In [1]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
import tensorflow as tf
import numpy as np
import model
import matplotlib.pyplot as plt
import pandas as pd


importing Jupyter notebook from model.ipynb


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
train_num_examples = 893231#总数：1116540
Iteration =13957
BATCH_SIZE = model.BATCH_SIZE
LEARNING_RATE_BASE = 0.005 # 初始学习率
LEARNING_RATE_DECAY = 0.99 # 学习率的衰减率
Sequence_index_train= 0
Sequence_index_test= 0
REGULARIZER = 0.0001
MOVING_AVERAGE_DECAY = 0.99 # 滑动平均的衰减率
width = 0.3

In [3]:
features_train = []
labels_train = []
Sequence_index_train = 0

In [4]:
def next_batch_train(Iteration,batch_size):  
    global features_train
    global labels_train
    global Sequence_index_train
    start_train = Sequence_index_train 
    Length_train = len(labels_train)
  #  print(Length_train)
    if Iteration == 0:
        Sequence_index_train = 0
        start_train = 0
        index_train = [ i for i in range(0,Length_train)]  
        np.random.shuffle(index_train)
        features_train = features_train[index_train]
        labels_train = labels_train[index_train]
#     if start_train + batch_size > Length_train: 
#         rest_num_examples = Length_train - start_train
#         features_rest_part = features_train[start_train:Length_train]
#         labels_rest_part = labels_train[start_train:Length_train]
#         index_train = [ i for i in range(0,Length_train) ]  
#         np.random.shuffle(index_train) 

#         features_train= features_train[index_train]
#         labels_train = labels_train[index_train]
#         start_strain = 0
#         Sequence_index_train = batch_size - rest_num_examples
#         end_train = Sequence_index_train
#         features_new_part = features_train[start_train:end_train]
#         labels_new_part = labels_train[start_train:end_train]

#         return np.concatenate((features_rest_part, features_new_part), axis=0), np.concatenate(
#                 (labels_rest_part, labels_new_part), axis=0)
#     else:
    Sequence_index_train += batch_size
    end_train = Sequence_index_train
#     print(start_train,end_train)
#     print(labels_train[start_train:end_train])
    return features_train[start_train:end_train],labels_train[start_train:end_train]
   

In [5]:
def train(features_train_,labels_train_):
    tf.reset_default_graph()
    features = tf.placeholder(tf.float32, [None,81])
    labels = tf.placeholder(tf.int32, [None])
    keep_prob = tf.placeholder(tf.float32)
    global_step = tf.Variable(0, trainable=False)# 声明一个全局计数器，并输出化为 0

    t_logits = model.inference(features,REGULARIZER,keep_prob)
    global features_train
    global labels_train
    features_train = features_train_
    labels_train = labels_train_
    
    # First, do a softmax on the output y of the last layer of the network, usually to find the probability that the output belongs to a certain class, which is actually a vector of size num_classes.
    # Then cross-entropy this vector with the actual label value. It should be noted that this function returns a vector.
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=t_logits+1e-10, labels=tf.reshape(labels,[-1])) 
    cem = tf.reduce_mean(ce) # Then average the obtaine d vector to get loss
    loss_re =tf.get_collection('losses')
    
    loss = cem + tf.add_n(tf.get_collection('losses')) # Add regularization losses
    
    # Realize an exponential reduction of the learning rate, which can make the model quickly approach the better solution in the early stages of training, and ensure that the model does not fluctuate too much in the later stages of training
    # Calculation formula: Decayed_learning_rate = learining_rate * decay_rate ^ (global_step / decay_steps)
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        train_num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)  # When staircase = True, (global_step / decay_steps) is converted to an integer to select different attenuation methods

    # # Pass in the learning rate, construct an optimizer that implements the gradient descent algorithm, and reduce the loss by using minimize to update the list of variables to be trained
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # To implement a moving average model, the parameter MOVING_AVERAGE_DECAY is used to control the speed of model updates. During the training process, a shadow variable is maintained for each variable, and the initial value of this shadow variable is maintained.
    # Is the initial value of the corresponding variable, each time the variable is updated, the shadow variable is updated accordingly

    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())  
    Accuracy_value_,nums,label_,logit_= model.accuracy(t_logits,tf.cast(labels,tf.int32))
    tf.summary.scalar('accuracy_value',Accuracy_value_)
    merged = tf.summary.merge_all()
    
    with tf.control_dependencies([train_step, ema_op]): # 将train_step 和ema_op 两个训练操作绑定到 train_op 上
        train_op = tf.no_op(name='train')
    saver = tf.train.Saver() # Instantiate a saver that saves and restores variables
  
    Accuracy_value = []
    Loss_value = []
    true_label = []
    pred_label = []   
    sess = tf.Session() # Create a session and manage it via a context manager in Python
    train_writer = tf.summary.FileWriter('model_folder/logs', sess.graph)
    init_op = tf.global_variables_initializer() # Initialize variables in calculation graphs
    sess.run(init_op)
    classes = ["BENIGN","ATTACK"]
    
    
    #     for Epo in range(0,EPOCHES):
    #         print('epoch为:',Epo)
    #         Accuracy_value_Epo = []
    for index in range(0,Iteration):   
        feature,label = next_batch_train(index,BATCH_SIZE)
        
        # _, loss_value_, accuracy_value,nums,reshape_logit_x,lable_x=session.run([t_optimizer, loss_value,Accuracy_value_,num,reshape_logits,lables], feed_dict={features:feature,labels:label,keep_prob:0.5})
        #_,loss_value_, accuracy_value,nums_,label_s,logit_s=session.run([t_optimizer,loss_value,Accuracy_value_,nums,label_,logit_],feed_dict={features:feature,labels:label,keep_prob:0.5})
        _, loss_value_, step,accuracy_value,nums_,label_s,logit_s = sess.run([train_op, loss, global_step,Accuracy_value_,nums,label_,logit_], feed_dict={features:feature,labels:label,keep_prob:0.5})
        # t_logit = session.run([t_logits],feed_dict={features:feature,labels:label,keep_prob:0.5})
        Accuracy_value.append(accuracy_value)
        #  Accuracy_value_Epo.append(accuracy_value)
        Loss_value.append(loss_value_)
        true_label.extend(label_s)
        pred_label.extend(logit_s)
       
        #batch_xs,batch_ys = mnist.train.next_batch(100)
        if index % 1000 == 0:
            print('index:', index, ' loss_value:', loss_value_, ' accuracy_value:', accuracy_value)
            # train_writer.add_summary(loss_value, index)
            #saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME), global_step=global_step)
            saver.save(sess, os.path.join('model_folder/saver', 'model.ckpt'))
    #sess.close()
    #sess = tf.InteractiveSession()
    op=tf.confusion_matrix(true_label,pred_label)
    confusion_matrix = sess.run(op)
    acc = model.accuracyofeachclass(confusion_matrix, true_label, num_classes = 2)
    print("Accuracy of two classes：",acc)
    Epoch_accracy=model.Epoch_accuracy_mean(Accuracy_value)
    # Calculate the accuracy of each category 
    # loss value
    #get_ipython().run_line_magic('matplotlib', 'notebook')
    #precision,accBA=Test_model.evaluation()     
    train_writer.close()
    #coord.request_stop()
    #coord.join(threads)
    return Accuracy_value,acc,Loss_value,Epoch_accracy